# IBM Streams sample application

This sample demonstrates creating a Streams Python application to perform some analytics, and viewing the results.

In this notebook, you'll see examples of how to :
 1. [Setup your data connections](#setup)
 2. [Create the application](#create)
 3. [Submit the application](#launch)
 4. [Connect to the running application to view data](#view)
 5. [Stop the application](#cancel)

# Overview

**About the sample**

This application simulates a data hub that receives readings from sensors. It computes the 30 second rolling average of the reported readings using [Pandas](https://pandas.pydata.org/).  

**How it works**
   
The Python application created in this notebook is submitted to the IBM Streams service for execution. Once the application is running in the service, you can connect to it from the notebook to retrieve the results.

<img src="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/04/how-it-works.jpg" alt="How it works">


### Documentation

- [Streams Python development guide](https://ibmstreams.github.io/streamsx.documentation/docs/latest/python/)
- [Streams Python API](https://streamsxtopology.readthedocs.io/)



<a name="setup"></a>
# 1. Setup
### 1.1 Add credentials for the IBM Streams service

With the cell below selected, click the "Connect to instance" button in the toolbar to insert the credentials for the service.

<a target="blank" href="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/02/connect_icp4d.gif">See an example</a>.

### 1.2 Verify `streamsx` package version

Run the cell below to check which version of the `streamsx` package is installed.  

If you need to upgrade, use

- `import sys`
- `!{sys.executable} -m pip install --user --upgrade streamsx` to upgrade the package.
- Or, use  `!{sys.executable} -m pip install --user streamsx==somever` to install a specific version of the package. 


In [ ]:
import streamsx.topology.context
print("INFO: streamsx package version: " + streamsx.topology.context.__version__)

#For more details uncomment line below.
#!pip show streamsx

<a id="create"></a>
# 2. Create the application
This application is going to ingest readings from simulated sensors and compute the 30 second rolling average value for each sensor.  

All Streams applications start with  a `Topology` object, so start by creating one:


In [ ]:
from streamsx.topology.topology import Topology

topo = Topology(name="SensorAverages")

## 2.1 Define sources
Your application needs some data to analyze, so the first step is to define a data source that produces the data being processed. 

Next, use the data source to create a `Stream` object. A `Stream` is a potentially infinite sequence of tuples containing the data to be analyzed.

Tuples are Python objects by default. Other supported formats include JSON. [See the doc for all supported formats](http://ibmstreams.github.io/streamsx.topology/doc/pythondoc/streamsx.topology.topology.html#stream).

### 2.1.1 Define a source class

Define a callable class that will produce the data to be analyzed.

This example class produces readings from sensors.

In [ ]:
import random 
import time
from datetime import datetime, timedelta

# Define a callable source 
class SensorReadingsSource(object):
    def __call__(self):
        # This is just an example of using generated data, 
        # Here you could connect to db
        # generate data
        # connect to data set
        # open file
        while True:
            time.sleep(0.001)
            sensor_id = random.randint(1,100)
            reading = {}
            reading ["sensor_id"] = "sensor_" + str(sensor_id)
            reading ["value"] =  random.random() * 3000
            reading["ts"] = int((datetime.now().timestamp())) 
            yield reading 

### 2.1.2  Create the `Stream `

Create a `Stream` called  `readings` that will contain the simulated data that `SensorReadingsSource` produces:

In [ ]:
#Create a stream from the data using Topology.source
readings = topo.source(SensorReadingsSource(), name="Readings")

# 2.2 Analyze data

Use a variety of methods in the `Stream` class to analyze your in-flight data, including applying machine learning models.

See the [common operations section](https://ibmstreams.github.io/streamsx.documentation/docs/python/1.6/python-appapi-devguide-4/) of the developer guide and the [documentation on the Stream class](https://ibmstreams.github.io/streamsx.topology/doc/pythondoc/streamsx.topology.topology.html#streamsx.topology.topology.Stream) for more details.


### 2.2.1 Filter data from the  `Stream`  

Use `Stream.filter()` to remove data that doesn't match a certain condition.

In [ ]:
# Accept only values greater than 100

valid_readings = readings.filter(lambda x : x["value"] > 100,
                                 name="ValidReadings")

# You could create another stream of the invalid data:
# invalid_readings = readings.filter(lambda x : x["value"] <= 100,)

### 2.2.2  Compute averages on the  `Stream`  

Define a function to compute the 30 second rolling average for the readings.

Steps are outlined in the code below.
See the [Window class documentation](http://ibmstreams.github.io/streamsx.topology/doc/pythondoc/streamsx.topology.topology.html#streamsx.topology.topology.Window)  for details.



In [ ]:
import pandas as pd

# 1. Define aggregation function
    
def average_reading(items_in_window):
    df = pd.DataFrame(items_in_window)
    readings_by_id = df.groupby("sensor_id")
    
    averages = readings_by_id["value"].mean()
    period_end = df["ts"].max()

    result = []
    for id, avg in averages.iteritems():
        result.append({"average": avg,
                "sensor_id": id,
                "period_end": time.ctime(period_end)})
               
    return result

# 2. Define window: e.g. a 30 second rolling average, updated every second

interval = timedelta(seconds=30)
window = valid_readings.last(size=interval).trigger(when=timedelta(seconds=1))


# 3. Pass aggregation function to Window.aggregate
# average_reading returns a list of the averages for each sensor,
# use flat map to convert it to individual tuples, one per sensor
rolling_average = window.aggregate(average_reading).flat_map()


# 2.3 Create a `View` to preview the tuples on the `Stream` 


A `View` is a connection to a `Stream` that becomes activated when the application is running. We examine the data from within the notebook in section 4, below.


In [ ]:
averages_view = rolling_average.view(name="RollingAverage", description="Sample of rolling averages for each sensor")

# 2.4 Define output

The `rolling_average` stream is our final result.  We will use `Stream.publish()` to make this stream available to other applications. 

If you want to send the stream to another database or system, you would use a sink function (similar to the source function) and invoke it using `Stream.for_each`.



In [ ]:
import json
# publish results as JSON
rolling_average.publish(topic="AverageReadings",
                        schema=json, 
                        name="PublishAverage")

# Other options include:
# invoke another sink function:
# rolling_average.for_each(func=send_to_db)

<a name="launch"></a>

# 3. Submit the application
A running Streams application is called a *job*. This next cell submits the application for execution and prints the resulting job id.

In [ ]:
from streamsx.topology import context

# Disable SSL certificate verification if necessary
cfg[context.ConfigParams.SSL_VERIFY] = False
# submit the topology 'topo'
submission_result = context.submit ("DISTRIBUTED", topo, config = cfg)

# The submission_result object contains information about the running application, or job
if submission_result.job:
    streams_job = submission_result.job
    print ("JobId: ", streams_job.id , "\nJob name: ", streams_job.name)

<a name="view"></a>

# 4. Use a `View` to access data from the job
Now that the job is started, use the `View` object you created in step 2.3 to start retrieving data from a `Stream`.

In [ ]:
# Connect to the view and display the data
queue = averages_view.start_data_fetch()
try:
    for val in range(10):
        print(queue.get())    
finally:
    averages_view.stop_data_fetch()

## 4.1 Display the results in real time
Calling `View.display()` from the notebook displays the results of the view in a table that is updated in real-time.

In [ ]:
# Display the results for 30 seconds
averages_view.display(duration=30)


## 4.2 See job status 

You can view job status and logs by going to **My Instances** > **Jobs**. Find your job based on the id printed above.
Retrieve job logs using the "Download logs" action from the job's context menu.

To view other information about the job such as detailed metrics, access the Streams Console.  Go to **My Instances** > **Provisioned Instances**. Select the Streams instance and open the URL listed under *externalConsoleEndpoint* or *serviceConsoleEndpoint*.

<a name="cancel"></a>

# 5. Cancel the job

This cell generates a widget you can use to cancel the job.

In [ ]:
#cancel the job in the IBM Streams service
submission_result.cancel_job_button()

You can also interact with the job through the [Job](https://streamsxtopology.readthedocs.io/en/stable/streamsx.rest_primitives.html#streamsx.rest_primitives.Job) object returned from `submission_result.job`

For example, use `job.cancel()` to cancel the running job directly.

# Summary

We started with a `Stream` called `readings`, which contained the data we wanted to analyze. Next, we used functions in the `Stream` object to perform simple analysis and produced the `rolling_average` stream.  This stream is `published` for other applications running within our Streams instance to access.

After submitting the application to the Streams service, we connected to the `rolling_average` view to see the results within the notebook.